### Prepare Repo

In [1]:
# !git clone https://github.com/facebookresearch/av_hubert.git

# %cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

# %cd fairseq
# !python -m pip install ./

In [2]:
%cd E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert

import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq")
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from transformers import Trainer, TrainingArguments
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn

E:\university\FYT\repos\multi_modal_ser\finetune_encoder\audio_video\av_hubert\avhubert


D:\anaconda\envs\py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]


In [3]:
def stacker(feats, stack_order):
    """
    Concatenating consecutive audio frames
    Args:
    feats - numpy.ndarray of shape [T, F]
    stack_order - int (number of neighboring frames to concatenate
    Returns:
    feats - numpy.ndarray of shape [T', F']
    """
    feat_dim = feats.shape[1]
    if len(feats) % stack_order != 0:
        res = stack_order - len(feats) % stack_order
        res = np.zeros([res, feat_dim]).astype(feats.dtype)
        feats = np.concatenate([feats, res], axis=0)
    feats = feats.reshape((-1, stack_order, feat_dim)).reshape(-1, stack_order*feat_dim)
    return feats

### Download Model

In [4]:
# !wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O E:/check_pts/avhubert.pt

### Build Model Pipeline

In [ ]:
user_dir = "E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "E:/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

In [ ]:
smp_id = "Ses01F_impro05_M017"
video_path = "E:/datasets/preprocessed/face_raw/{}/{}.mp4".format(smp_id.split("_")[0][:-1],smp_id)
audio_path = "E:/datasets/preprocessed/spectrogram/raw/{}.wav".format(smp_id)

# Load Video
# Mute transform first
transform = avhubert_utils.Compose([
  avhubert_utils.Normalize(0.0, 255.0),
  avhubert_utils.CenterCrop((task.cfg.image_crop_size, task.cfg.image_crop_size)),
  avhubert_utils.Normalize(task.cfg.image_mean, task.cfg.image_std)])

frames = avhubert_utils.load_video(video_path)
# frames = transform(frames)
frames = torch.FloatTensor(frames).unsqueeze(dim=0).unsqueeze(dim=0)
video_feats = frames

# Load Audio
sample_rate, wav_data = wavfile.read(audio_path)
in_features = model.feature_extractor_audio.proj.in_features
assert sample_rate == 16_000 and len(wav_data.shape) == 1
audio_feats = logfbank(wav_data, samplerate=sample_rate).astype(np.float32) # [T, F]
audio_feats = stacker(audio_feats, in_features//26) # [T/stack_order_audio, F*stack_order_audio]

# Match Audio Video
print(audio_feats.shape)
print(video_feats.shape)

diff = audio_feats.shape[0] - video_feats.shape[2]
if diff < 0:
    audio_feats = np.concatenate([audio_feats, np.zeros([-diff, audio_feats.shape[-1]], dtype=audio_feats.dtype)])
elif diff > 0:
    audio_feats = audio_feats[:-diff]

audio_feats = torch.from_numpy(audio_feats.astype(np.float32)).T
audio_feats = F.layer_norm(audio_feats, audio_feats.shape[1:])
audio_feats = audio_feats.unsqueeze(dim=0)


In [7]:
model.eval()
with torch.no_grad():
    # Specify output_layer if you want to extract feature of an intermediate layer
    feature, _ = model.extract_finetune(source={'video': video_feats , 
                                                'audio': audio_feats}, padding_mask=None, output_layer=None)
    feature = feature.squeeze(dim=0) 
print(f"Video feature shape: {feature.shape}")

Video feature shape: torch.Size([192, 768])


In [8]:
### Save Model Standalone Checkpoints

In [31]:
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/avhubert_ds.pt")
print(type(mmser_ds))

<class 'avhubert_ds.AVHUBERTDataset'>


In [32]:
print(mmser_ds[2:4]['audio'].shape)
print(mmser_ds[2:4]['video'].shape)
print(mmser_ds[2:4]['padding_mask'].shape)

torch.Size([2, 500, 104])
torch.Size([2, 1, 500, 88, 88])
torch.Size([2, 500, 104])


In [33]:
print(mmser_ds[24]['audio'].shape)
print(mmser_ds[24]['video'].shape)
print(mmser_ds[24]['padding_mask'].shape)

torch.Size([500, 104])
torch.Size([1, 500, 88, 88])
torch.Size([500, 104])


In [34]:
import os
AUDIORATE = 16000

outputs = model.extract_finetune(mmser_ds[:2])
outputs

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [2, 500] but got: [2, 104].

In [14]:
outputs[0].shape

torch.Size([2, 500, 768])

In [15]:
attention_mask = mmser_ds[:4]["padding_mask"]

In [16]:
def forward_padding_mask(
        features: torch.Tensor, padding_mask: torch.Tensor,
    ):
        extra = padding_mask.size(1) % features.size(1)
        if extra > 0:
            padding_mask = padding_mask[:, :-extra]
        padding_mask = padding_mask.view(
            padding_mask.size(0), features.size(1), -1
        )
        padding_mask = padding_mask.all(-1)
        return padding_mask

In [17]:
forwarded_attention_mask = forward_padding_mask(outputs[0], attention_mask)

### Define the model

In [18]:
class AVHUBERTClassifier(torch.nn.Module):

    def __init__(self, encoder, hidden_size, proj_size, num_labels=4):
        super(AVHUBERTClassifier, self).__init__()
        self.encoder = encoder
        self.num_labels = num_labels
        self.hidden_size = hidden_size
        self.proj_size = proj_size
        self.num_labels = num_labels
        
        self.projector = nn.Linear(self.hidden_size, self.proj_size)
        self.classifier = nn.Linear(self.proj_size, num_labels)

    def forward(
        self,
        audio, video, padding_mask, **kwargs
    ):
        with torch.no_grad():
            outputs = self.encoder.extract_finetune(
                {
                    "audio": audio, 
                    "video": video,
                    "padding_mask": padding_mask
                }
            )
            hidden_states = outputs[0]
        
        hidden_states = self.projector(hidden_states)
        attention_mask = padding_mask
        padding_mask = forward_padding_mask(hidden_states, attention_mask)
        hidden_states[~padding_mask] = 0.0
        pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)
        logits = self.classifier(pooled_output)
        return {"logits":logits}
    

In [19]:
classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
classifier(**mmser_ds[:4])

{'logits': tensor([[-0.0074,  0.0165, -0.1072, -0.0366],
         [-0.0094,  0.0202, -0.1158, -0.0336],
         [ 0.0008,  0.0165, -0.0964, -0.0407],
         [ 0.0049,  0.0226, -0.0782, -0.0437]], grad_fn=<AddmmBackward0>)}

In [20]:
def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

In [21]:
meta_df_ = mmser_ds.df_
sess_dict = meta_df_.groupby("session").groups
all_indices = set(meta_df_.index.tolist())
sess_ds = {}
for sess in sess_dict:
#     sess_ds[sess+"_train"] = Subset(mmser_ds, 
#                                     indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    
    
    
SESS_ID = list(sess_dict.keys())[0]
train_set, val_set, test_set = build_ds(SESS_ID)

Train Samples: 380
Val Samples: 127
Test Samples: 507


In [22]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").type(torch.LongTensor).to(device)
        inputs = {"video": inputs["video"], "audio":inputs["audio"], "padding_mask":inputs["padding_mask"]}
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss() 
        loss = loss_fct(logits.view(-1, self.model.num_labels), labels.view(-1))        
        return (loss, outputs) if return_outputs else loss
    
def weighted_acc(y_true, y_pred):
    return np.sum((np.array(y_pred).ravel() == np.array(y_true).ravel()))*1.0/len(y_true)
    
def unweighted_acc(y_true, y_pred):
    y_true = np.array(y_true).ravel()
    y_pred = np.array(y_pred).ravel()
    classes = np.unique(y_true)
    classes_accuracies = np.zeros(classes.shape[0])
    for num, cls in enumerate(classes):
        classes_accuracies[num] = weighted_acc(y_true[y_true == cls], y_pred[y_true == cls])
    return np.mean(classes_accuracies)

def compute_metrics(eval_preds):
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)

    metric_f1 = load_metric("f1")
    metric_acc = load_metric("accuracy")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    f1_ = metric_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    acc_ = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    
    return {"wa":weighted_acc(labels, predictions), 
            "ua":unweighted_acc(labels, predictions),
            "f1":f1_, 
            "accuracy":acc_}

In [23]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=2)
# next(iter(train_loader))

In [24]:
output_dir=os.path.join("avhubert")

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = 3 # int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = 3 #int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=1e-3
training_args.num_train_epochs=50

trainer = CustomTrainer(
    model=classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

D:\anaconda\envs\py37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brucehu. Use `wandb login --relogin` to force relogin


KeyboardInterrupt: 